In [1]:
# For data processing
import numpy as np
from math import sqrt
# For data processing and manipulation
import pandas as pd
import csv
 
# For date calculations
import datetime
import time
 
# For ploting data
import IPython
import IPython.display
 
import itertools
from itertools import cycle
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
 
# For checking path
import os , gc
from os import path
import csv
import json
from IPython.display import clear_output 
 
from scipy.stats import hmean
 
from sklearn.metrics import mean_squared_error
import requests

import ipywidgets as widgets
from ipywidgets import interact, interact_manual


In [2]:
from pymongo import MongoClient
# pprint library is used to make the output look more pretty
'''
Should move db code to other file
'''
# connect to MongoDB, change the << MONGODB URL >> to reflect your own connection string
client = MongoClient(port=27017)
db = client.test

In [3]:
ROOTPATH = 'C:\\Study\\CSE 4000\\New folder\\Thesis\\'
#PATH_IMAGE = ROOTPATH+'images_chicago\\'
#PATH_WEAT_CHI = ROOTPATH+"dataset\\processed_weather_data_chicago_2004_2017.csv"

test query

In [7]:
requestData = {
    'wx_phrase': 'Mostly Cloudy',
    'pressure': 1003.99}
#urlTest = DATABASE_URL + "/query/find/chicago/weather"
#response = requests.get(urlTest, json=requestData)
response = db.weather.philadelphia.find(requestData)

#do something with res
print(response)
"""
if response.ok:
    try:
        data = response.json()
        print(len(data))
    except json.JSONDecodeError as e:
        print('Error in decoding json: ' + e)
    else:  
        print("Error:", response.status_code)"""

'\nif response.ok:\n    try:\n        data = response.json()\n        print(len(data))\n    except json.JSONDecodeError as e:\n        print(\'Error in decoding json: \' + e)\n    else:  \n        print("Error:", response.status_code)'

In [71]:
del response

<h1> Columns </h1>

In [8]:
# Record the start time
start_time = time.time()

# Get a single document from the collection
sample_document = db.weather.philadelphia.find_one()

# Extract field names from the document
field_names = list(sample_document.keys())

print("Field Names:", field_names)
print("Number of Fields:", len(field_names))

# Record the end time
end_time = time.time()
execution_time = end_time - start_time
print("Execution Time: {:.2f} seconds".format(execution_time))


Field Names: ['_id', '', 'expire_time_gmt', 'valid_time_gmt', 'temp', 'wx_phrase', 'heat_index', 'rh', 'pressure', 'vis', 'wc', 'wdir', 'wspd', 'max_temp', 'min_temp', 'expire_date_time', 'valid_date_time']
Number of Fields: 17
Execution Time: 0.00 seconds


In [73]:
del sample_document, field_names

<h3><b> Phrase Types without Pipeline </h3></b>

In [10]:
# Record the start time
start_time = time.time()

requestData = {
 'field': 'wx_phrase'}

#urlTest = DATABASE_URL + "/query/unique/chicago/weather"
#x = requests.get(urlTest, json=requestData)
distinct_wx_phrases = db.weather.philadelphia.distinct('wx_phrase')
# Print the distinct values
for phrase in distinct_wx_phrases:
    print(phrase)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print execution time
print("Execution Time: {:.2f} seconds".format(execution_time))

Blowing Snow / Windy
Cloudy
Cloudy / Windy
Drizzle
Drizzle and Fog
Fair
Fair / Windy
Fog
Haze
Haze / Windy
Heavy Rain
Heavy Rain / Windy
Heavy Sleet / Windy
Heavy Snow
Heavy Snow / Windy
Heavy T-Storm
Heavy T-Storm / Windy
Light Drizzle
Light Drizzle / Windy
Light Freezing Drizzle
Light Freezing Rain
Light Freezing Rain / Windy
Light Rain
Light Rain / Windy
Light Rain with Thunder
Light Sleet
Light Sleet / Windy
Light Snow
Light Snow / Freezing Rain
Light Snow / Windy
Light Snow and Sleet
Light Snow and Sleet / Windy
Mist
Mostly Cloudy
Mostly Cloudy / Windy
Partly Cloudy
Partly Cloudy / Windy
Patches of Fog
Rain
Rain / Freezing Rain
Rain / Windy
Rain and Sleet
Rain and Sleet / Windy
Rain and Snow
Rain and Snow / Windy
Shallow Fog
Sleet
Snow
Snow / Windy
Snow and Sleet
Snow and Sleet / Windy
Squalls / Windy
T-Storm
T-Storm / Windy
Thunder
Thunder / Windy
Thunder in the Vicinity
Wintry Mix
Wintry Mix / Windy
Execution Time: 0.17 seconds


In [11]:
del distinct_wx_phrases

<h3><b> Phrase Types using Pipeline </h3></b>

In [12]:
# Record the start time
start_time = time.time()

phraseXCount = {}
#urlTest = DATABASE_URL + "/query/aggregate/chicago/weather"
pipeline = [
    {
        "$group": {
                "_id": "$wx_phrase",
                "Count" : {"$sum" : 1}
        }
    }
]
# Run the aggregation query
data_cursor = db.weather.philadelphia.aggregate(pipeline)

# Convert cursor to list (if dataset is large, consider processing each document in a loop instead)
data = list(data_cursor)

# Initialize variables
phraseXCount = {}
tsum = sum(doc['Count'] for doc in data)
c = 0

# Process each document
for doc in data:
    print(doc)
    phraseXCount[doc['_id']] = doc['Count']
    c += 1

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print results
print("Total Unique Types =", c)
print("Total Cases =", tsum)
print("Execution Time: {:.2f} seconds".format(execution_time))

{'_id': 'Light Snow / Freezing Rain', 'Count': 2}
{'_id': 'Heavy Sleet / Windy', 'Count': 2}
{'_id': 'Heavy T-Storm / Windy', 'Count': 73}
{'_id': 'Partly Cloudy / Windy', 'Count': 1136}
{'_id': 'Rain and Sleet / Windy', 'Count': 1}
{'_id': 'Drizzle', 'Count': 5}
{'_id': 'Heavy T-Storm', 'Count': 258}
{'_id': 'Sleet', 'Count': 5}
{'_id': 'Heavy Snow', 'Count': 34}
{'_id': 'Thunder / Windy', 'Count': 27}
{'_id': 'Rain / Freezing Rain', 'Count': 37}
{'_id': 'Light Drizzle / Windy', 'Count': 25}
{'_id': 'Rain and Snow', 'Count': 25}
{'_id': 'Light Freezing Drizzle', 'Count': 23}
{'_id': 'Cloudy / Windy', 'Count': 2923}
{'_id': 'Light Snow and Sleet', 'Count': 38}
{'_id': 'Thunder in the Vicinity', 'Count': 48}
{'_id': 'Haze', 'Count': 262}
{'_id': 'Squalls / Windy', 'Count': 1}
{'_id': 'Shallow Fog', 'Count': 8}
{'_id': 'Snow and Sleet / Windy', 'Count': 9}
{'_id': 'Light Freezing Rain / Windy', 'Count': 7}
{'_id': 'Heavy Rain / Windy', 'Count': 99}
{'_id': 'Patches of Fog', 'Count': 46}


<h3><b> working Average Temprature</h3></b>

In [13]:
# Record the start time
start_time = time.time()

#urlTest = DATABASE_URL + "/query/aggregate/chicago/weather"
"""pipeline = [
    {
        "$group": {
            "_id": "$temp",
            "total_temp": {"$sum": 1},
        }
    },
    {
        "$group": {
            "_id": None,
            "average_temp_per_year": {"$avg": "$total_temp"},
        }
    }
]"""
pipeline = [
    {
        "$group": {
            "_id": None,
            "avg_temperature": { "$avg": "$temp" }
        }
    }
]
# Run the aggregation query
data_cursor = db.weather.philadelphia.aggregate(pipeline)

# Convert cursor to list (if dataset is large, consider processing each document in a loop instead)
data = list(data_cursor)

# Print the result
for doc in data:
    print(doc)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print execution time
print("Execution Time: {:.2f} seconds".format(execution_time))
        

{'_id': None, 'avg_temperature': 13.809916160740098}
Execution Time: 0.08 seconds


<h3><b> Max Temprature </h3></b>

In [14]:
# Record the start time
start_time = time.time()

#urlTest = DATABASE_URL + "/query/aggregate/chicago/weather"
pipeline = [
    {
        "$group": {
            "_id": None,
            "max_temperature": { "$max": "$temp" }
        }
    }
]

# Run the aggregation query
data_cursor = db.weather.philadelphia.aggregate(pipeline)

# Convert cursor to list (if dataset is large, consider processing each document in a loop instead)
data = list(data_cursor)

# Print the result
for doc in data:
    print(doc)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print execution time
print("Execution Time: {:.2f} seconds".format(execution_time))

{'_id': None, 'max_temperature': 40.0}
Execution Time: 0.07 seconds


<h3><b> 6.2.3 working min Temprature </h3></b>

In [15]:
# Record the start time
start_time = time.time()

#rlTest = DATABASE_URL + "/query/aggregate/chicago/weather"
pipeline = [
    {
        "$group": {
            "_id": None,
            "min_temperature": { "$min": "$temp" }
        }
    }
]

# Run the aggregation query
data_cursor = db.weather.philadelphia.aggregate(pipeline)

# Process each document in the cursor
for doc in data_cursor:
    print(doc)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print execution time
print("Execution Time: {:.2f} seconds".format(execution_time))
        

{'_id': None, 'min_temperature': -17.0}
Execution Time: 0.07 seconds


<h3><b>working max temp per year </h3></b>

In [16]:
# Record the start time
start_time = time.time()

#urlTest = DATABASE_URL + "/query/aggregate/chicago/weather"

# Aggregation pipeline
pipeline = [
    {
        "$addFields": {
            "expire_date_time": {
                "$dateFromString": {
                    "dateString": "$expire_date_time",
                    "format": "%d/%m/%Y %H:%M:%S"  # Adjusted date format here
                }
            }
        }
    },
    {
        "$addFields": {
            "year": {"$year": "$expire_date_time"}
        }
    },
    {
        "$group": {
            "_id": "$year",
            "max_temperature": {"$max": "$temp"}
        }
    },
    {
        "$sort": {
            "_id": 1
        }
    }
]

# Run the aggregation query
data_cursor = db.weather.philadelphia.aggregate(pipeline)

# Stream process each document in the cursor
for doc in data_cursor:
    print(doc)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print execution time
print("Execution Time: {:.2f} seconds".format(execution_time))

{'_id': 2004, 'max_temperature': 33.0}
{'_id': 2005, 'max_temperature': 36.0}
{'_id': 2006, 'max_temperature': 36.0}
{'_id': 2007, 'max_temperature': 36.0}
{'_id': 2008, 'max_temperature': 36.0}
{'_id': 2009, 'max_temperature': 35.0}
{'_id': 2010, 'max_temperature': 40.0}
{'_id': 2011, 'max_temperature': 39.0}
{'_id': 2012, 'max_temperature': 38.0}
{'_id': 2013, 'max_temperature': 36.0}
{'_id': 2014, 'max_temperature': 35.0}
{'_id': 2015, 'max_temperature': 36.0}
{'_id': 2016, 'max_temperature': 36.0}
{'_id': 2017, 'max_temperature': 35.0}
{'_id': 2018, 'max_temperature': 36.0}
{'_id': 2019, 'max_temperature': 37.0}
{'_id': 2020, 'max_temperature': 36.0}
{'_id': 2021, 'max_temperature': 36.0}
{'_id': 2022, 'max_temperature': 37.0}
Execution Time: 0.44 seconds


<h1><b>6.1.5 Count Rainy Days </h1></b>

In [11]:
# Record the start time
start_time = time.time()
import re
match_word = "Rain"
# Aggregation pipeline to match wx_phrase with 'Rain' (case insensitive)
pipeline = [
    {
        "$match": {
            "wx_phrase": {"$regex": match_word, "$options": "i"}
        }
    }
]

# Run the aggregation query
data_cursor = db.weather.philadelphia.aggregate(pipeline)

# Initialize count
c = 0
rainXcount = {}

# Stream process each document in the cursor
for doc in data_cursor:
    rainXcount[doc['_id']] = doc['expire_date_time']
    print(f"On date : {doc['expire_date_time']} , {doc['wx_phrase']} was observed")
    c += 1

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print results
print("Total Cases =", c)
print("Execution Time: {:.2f} seconds".format(execution_time))

On date : 20/03/2014 02:00:00 , Light Rain was observed
On date : 20/03/2014 02:00:00 , Light Rain was observed
On date : 20/03/2014 02:00:00 , Light Rain was observed
On date : 20/03/2014 02:00:00 , Light Rain was observed
On date : 20/03/2014 03:00:00 , Light Rain was observed
On date : 20/03/2014 03:00:00 , Light Rain was observed
On date : 20/03/2014 03:00:00 , Light Rain was observed
On date : 20/03/2014 04:00:00 , Light Rain was observed
On date : 20/03/2014 04:00:00 , Light Rain was observed
On date : 20/03/2014 04:00:00 , Light Rain was observed
On date : 20/03/2014 04:00:00 , Light Rain was observed
On date : 20/03/2014 05:00:00 , Light Rain was observed
On date : 20/03/2014 05:00:00 , Light Rain was observed
On date : 20/03/2014 05:00:00 , Light Rain was observed
On date : 20/03/2014 06:00:00 , Light Rain was observed
On date : 20/03/2014 06:00:00 , Rain was observed
On date : 20/03/2014 06:00:00 , Light Rain was observed
On date : 20/03/2014 06:00:00 , Light Rain was observe

Wind Direction Distribution:



In [18]:
# Record the start time
start_time = time.time()
# Aggregation pipeline for wind direction count
pipeline = [
    {
        "$group": {
            "_id": "$wdir",
            "Intensity": {"$sum": 1}
        }
    },
    {
        "$sort": {
            "_id": 1
        }
    }
]

# Run the aggregation query
data_cursor = db.weather.philadelphia.aggregate(pipeline)

# Initialize variables
wdirXCount = {}
tsum = 0
c = 0

# Stream process each document in the cursor
for doc in data_cursor:
    print(doc)
    wdirXCount[doc['_id']] = doc['Intensity']
    tsum += doc['Intensity']
    c += 1

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print results
print("Total directions =", c)
print("Total Cases =", tsum)
print("Execution Time: {:.2f} seconds".format(execution_time))

{'_id': None, 'Intensity': 21299}
{'_id': 10.0, 'Intensity': 3897}
{'_id': 20.0, 'Intensity': 2356}
{'_id': 30.0, 'Intensity': 3445}
{'_id': 40.0, 'Intensity': 6497}
{'_id': 50.0, 'Intensity': 7563}
{'_id': 60.0, 'Intensity': 6252}
{'_id': 70.0, 'Intensity': 6220}
{'_id': 80.0, 'Intensity': 5324}
{'_id': 90.0, 'Intensity': 4154}
{'_id': 100.0, 'Intensity': 3378}
{'_id': 110.0, 'Intensity': 2833}
{'_id': 120.0, 'Intensity': 2555}
{'_id': 130.0, 'Intensity': 2450}
{'_id': 140.0, 'Intensity': 2563}
{'_id': 150.0, 'Intensity': 2666}
{'_id': 160.0, 'Intensity': 3405}
{'_id': 170.0, 'Intensity': 4101}
{'_id': 180.0, 'Intensity': 4976}
{'_id': 190.0, 'Intensity': 7393}
{'_id': 200.0, 'Intensity': 8036}
{'_id': 210.0, 'Intensity': 6652}
{'_id': 220.0, 'Intensity': 7347}
{'_id': 230.0, 'Intensity': 9727}
{'_id': 240.0, 'Intensity': 6736}
{'_id': 250.0, 'Intensity': 6590}
{'_id': 260.0, 'Intensity': 6149}
{'_id': 270.0, 'Intensity': 4647}
{'_id': 280.0, 'Intensity': 5972}
{'_id': 290.0, 'Intensi

in a range check

In [19]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
    {
        "$match": {
            "$and": [
                {"pressure": {"$gt": 1000.00}},
                {"pressure": {"$lt": 1001.00}}
            ]
        }
    }
]

# Run the aggregation query
data = db.weather.philadelphia.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")

#.087 seconds
#{"pressure": {"$gt": 1000.00, "$lt": 1001.00}}

,_id,,expire_time_gmt,valid_time_gmt,temp,wx_phrase,heat_index,rh,pressure,vis,wc,wdir,wspd,expire_date_time,valid_date_time,max_temp,min_temp
0,65c7b6cea264af1ce4e95c4a,454,1074448440,1074441240,1.0,Cloudy,1.0,100.0,1000.23,3.0,1.0,220.0,6.0,19/01/2004 00:00:00,18/01/2004 22:00:00,NaN,NaN
1,65c7b6cea264af1ce4e95c5a,470,1074488040,1074480840,1.0,Cloudy / Windy,1.0,56.0,1000.93,16.0,-7.0,320.0,43.0,19/01/2004 11:00:00,19/01/2004 09:00:00,NaN,NaN
2,65c7b6cea264af1ce4e95e96,1042,1076154840,1076147640,2.0,Cloudy,2.0,100.0,1000.23,16.0,2.0,300.0,6.0,07/02/2004 18:00:00,07/02/2004 16:00:00,NaN,NaN
3,65c7b6cea264af1ce4e95fe9,1381,1077342840,1077335640,4.0,Cloudy,4.0,93.0,1000.63,11.0,1.0,60.0,13.0,21/02/2004 12:00:00,21/02/2004 10:00:00,NaN,NaN
4,65c7b6cea264af1ce4e95fea,1382,1077346440,1077339240,3.0,Mostly Cloudy,3.0,93.0,1000.03,11.0,1.0,60.0,7.0,21/02/2004 13:00:00,21/02/2004 11:00:00,10.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1513,65c7b6d5a264af1ce4ec9d5b,213719,1669587840,1669580640,13.0,Cloudy,13.0,93.0,1000.44,16.0,13.0,230.0,19.0,28/11/2022 04:00:00,28/11/2022 02:00:00,NaN,NaN
1514,65c7b6d5a264af1ce4ec9d71,213741,1669629240,1669622040,11.0,Mostly Cloudy,11.0,83.0,1000.44,16.0,11.0,250.0,11.0,28/11/2022 16:00:00,28/11/2022 14:00:00,NaN,NaN
1515,65c7b6d5a264af1ce4ec9d72,213742,1669632840,1669625640,12.0,Mostly Cloudy,12.0,75.0,1000.44,16.0,12.0,220.0,17.0,28/11/2022 17:00:00,28/11/2022 15:00:00,NaN,NaN
1516,65c7b6d5a264af1ce4ec9d73,213743,1669636440,1669629240,10.0,Partly Cloudy,10.0,83.0,1000.78,16.0,10.0,240.0,15.0,28/11/2022 18:00:00,28/11/2022 16:00:00,NaN,NaN


Execution Time: 0.09 seconds


<h1><b> Complex Queries </h1></b>

<h3> total sum of temperatures and pressures for each distinct weather phrase (wx_phrase) </h3>

In [20]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
    {
        "$group": {
            "_id": "$wx_phrase",
            "total_temp": {"$sum": "$temp"},
            "total_pressure": {"$sum": "$pressure"}
        }
    },
    {
        "$sort": {"_id": 1}
    },
    {
        "$project": {
            "_id": 0,
            "wx_phrase": "$_id",
            "total_temp": 1,
            "total_pressure": 1
        }
    }
]

# Run the aggregation query
data = db.weather.philadelphia.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")
#.186 seconds

,total_temp,total_pressure,wx_phrase
0,479.0,30693.26,None
1,-24.0,3011.78,Blowing Snow / Windy
2,938002.0,75296171.85,Cloudy
3,30917.0,2935855.50,Cloudy / Windy
4,73.0,5086.83,Drizzle
5,44.0,4004.94,Drizzle and Fog
6,497908.0,41927001.11,Fair
7,6630.0,1316675.03,Fair / Windy
8,23986.0,1994749.69,Fog
9,4240.0,266552.11,Haze


Execution Time: 0.11 seconds


<h3> Daily Data</h3>

In [21]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
    {
        "$addFields": {
            "day": {
                "$dateFromString": {
                    "dateString": "$valid_date_time",
                    "format": "%d/%m/%Y %H:%M:%S"  # Include time in the format
                }
            }
        }
    },
    {
        "$group": {
            "_id": {
                "day": {
                    "$dateToString": { "format": "%Y-%m-%d", "date": "$day" }
                }
            },
            "avg_temp": {"$avg": "$temp"},
            "total_wspd": {"$sum": "$wspd"},
            "max_visibility": {"$max": "$vis"},
            "distinct_weather_conditions": {"$addToSet": "$wx_phrase"}
        }
    },
    {
        "$addFields": {
            "count_distinct_weather_conditions": {"$size": "$distinct_weather_conditions"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "day": "$_id.day",
            "avg_temp": 1,
            "total_wspd": 1,
            "max_visibility": 1,
            "distinct_weather_conditions": "$count_distinct_weather_conditions"
        }
    },
    {
        "$sort": {"day": 1}
    }
]


# Run the aggregation query
data = db.weather.philadelphia.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")
#0.13 seconds

,avg_temp,total_wspd,max_visibility,day,distinct_weather_conditions
0,3.076923,210.0,16.0,2004-01-01,3
1,5.208333,242.0,16.0,2004-01-02,2
2,7.200000,220.0,16.0,2004-01-03,3
3,9.925926,215.0,16.0,2004-01-04,2
4,5.916667,609.0,16.0,2004-01-05,1
...,...,...,...,...,...
6928,-1.375000,240.0,16.0,2022-12-27,3
6929,0.541667,138.0,16.0,2022-12-28,4
6930,3.125000,185.0,16.0,2022-12-29,3
6931,4.846154,150.0,16.0,2022-12-30,2


Execution Time: 0.59 seconds


<h1> <b> addition </b></h1>

<h1> <b> subtraction </b></h1>

<h1> <b> comparing </b></h1>

<h3> 6.1.1 Selecting records with temperature differences within a specified range and wind speed above a certain threshold </h3>

In [5]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
    {
        "$addFields": {
            "temperature_difference": {
                "$subtract": ["$max_temp", "$min_temp"]
            },
            "wind_speed_category": {
                "$cond": {
                    "if": { "$gt": ["$wspd", 20] },
                    "then": "High wind",
                    "else": "Low wind"
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "Unnamed: 0": 1,
            "expire_time_gmt": 1,
            "valid_time_gmt": 1,
            "temp": 1,
            "wspd": 1,
            "wx_phrase": 1,
            "valid_date_time": 1,
            "expire_date_time": 1,
            "temperature_difference": 1,
            "wind_speed_category": 1
        }
    }
]

# Run the aggregation query
data = db.weather.philadelphia.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")
#0.144 seconds
#.2323 seconds
"""{"_id":0,"Unnamed: 0": 1, "expire_time_gmt": 1,
            "valid_time_gmt": 1,
            "temp": 1,
            "wspd": 1,"wx_phrase": 1, "valid_date_time": 1, "expire_date_time": 1, "temperature_difference": {"$subtract": ["$max_temp", "$min_temp"]}, "wind_speed_category": {"$cond": {"if": {"$gt": ["$wspd", 20]}, "then": "High wind", "else": "Low wind"}}}"""

,expire_time_gmt,valid_time_gmt,temp,wx_phrase,wspd,expire_date_time,valid_date_time,temperature_difference,wind_speed_category
0,1072940040,1072932840,3.0,Fair,17.0,01/01/2004 13:00:00,01/01/2004 11:00:00,10.0,Low wind
1,1072943640,1072936440,1.0,Fair,11.0,01/01/2004 14:00:00,01/01/2004 12:00:00,5.0,Low wind
2,1072947240,1072940040,1.0,Fair,13.0,01/01/2004 15:00:00,01/01/2004 13:00:00,NaN,Low wind
3,1072950840,1072943640,0.0,Fair,13.0,01/01/2004 16:00:00,01/01/2004 14:00:00,NaN,Low wind
4,1072954440,1072947240,1.0,Fair,11.0,01/01/2004 17:00:00,01/01/2004 15:00:00,NaN,Low wind
...,...,...,...,...,...,...,...,...,...
214733,1672455240,1672448040,6.0,Mostly Cloudy,6.0,31/12/2022 09:00:00,31/12/2022 07:00:00,NaN,Low wind
214734,1672458840,1672451640,6.0,Mostly Cloudy,0.0,31/12/2022 10:00:00,31/12/2022 08:00:00,NaN,Low wind
214735,1672462440,1672455240,7.0,Mostly Cloudy,0.0,31/12/2022 11:00:00,31/12/2022 09:00:00,NaN,Low wind
214736,1672466040,1672458840,8.0,Mostly Cloudy,0.0,31/12/2022 12:00:00,31/12/2022 10:00:00,NaN,Low wind


Execution Time: 1.09 seconds


'{"_id":0,"Unnamed: 0": 1, "expire_time_gmt": 1,\n            "valid_time_gmt": 1,\n            "temp": 1,\n            "wspd": 1,"wx_phrase": 1, "valid_date_time": 1, "expire_date_time": 1, "temperature_difference": {"$subtract": ["$max_temp", "$min_temp"]}, "wind_speed_category": {"$cond": {"if": {"$gt": ["$wspd", 20]}, "then": "High wind", "else": "Low wind"}}}'

<h3> 6.1.2 Identifying records where visibility is within a certain range and the temperature is below freezing </h3>

In [23]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
    {
        "$addFields": {
            "wind_direction": {
                "$cond": {
                    "if": {
                        "$and": [
                            {"$gte": ["$wdir", 0]},
                            {"$lte": ["$wdir", 180]}
                        ]
                    },
                    "then": "Easterly Wind",
                    "else": "Westerly Wind"
                }
            },
            "temperature_status": {
                "$cond": {
                    "if": {"$lt": ["$temp", 0]},
                    "then": "Below freezing",
                    "else": "Above freezing"
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "Unnamed: 0": 1,
            "expire_date_time": 1,
            "expire_time_gmt": 1,
            "valid_date_time": 1,
            "valid_time_gmt": 1,
            "vis": 1,
            "wx_phrase": 1,
            "wind_direction": 1,
            "temp": 1,
            "temperature_status": 1
        }
    }
]

# Run the aggregation query
data = db.weather.philadelphia.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")
#0.137 seconds
#.2243
"""{"_id": 0,
            "Unnamed: 0": 1,
            "expire_date_time": 1,
            "expire_time_gmt": 1,
            "valid_date_time": 1,
            "valid_time_gmt": 1,
            "vis": 1,
            "wx_phrase": 1, "wind_direction": { "$cond": { "if": { "$and": [{ "$gte": ["$wdir", 0] }, { "$lte": ["$wdir", 180] }] }, "then": "Easterly Wind", "else": "Westerly Wind" } }, "temperature_status": { "$cond": { "if": { "$lt": ["$temp", 0] }, "then": "Below freezing", "else": "Above freezing" } }}"""

,expire_time_gmt,valid_time_gmt,temp,wx_phrase,vis,expire_date_time,valid_date_time,wind_direction,temperature_status
0,1072940040,1072932840,3.0,Fair,16.0,01/01/2004 13:00:00,01/01/2004 11:00:00,Westerly Wind,Above freezing
1,1072943640,1072936440,1.0,Fair,16.0,01/01/2004 14:00:00,01/01/2004 12:00:00,Westerly Wind,Above freezing
2,1072947240,1072940040,1.0,Fair,16.0,01/01/2004 15:00:00,01/01/2004 13:00:00,Westerly Wind,Above freezing
3,1072950840,1072943640,0.0,Fair,16.0,01/01/2004 16:00:00,01/01/2004 14:00:00,Westerly Wind,Above freezing
4,1072954440,1072947240,1.0,Fair,16.0,01/01/2004 17:00:00,01/01/2004 15:00:00,Westerly Wind,Above freezing
...,...,...,...,...,...,...,...,...,...
214733,1672455240,1672448040,6.0,Mostly Cloudy,16.0,31/12/2022 09:00:00,31/12/2022 07:00:00,Easterly Wind,Above freezing
214734,1672458840,1672451640,6.0,Mostly Cloudy,16.0,31/12/2022 10:00:00,31/12/2022 08:00:00,Westerly Wind,Above freezing
214735,1672462440,1672455240,7.0,Mostly Cloudy,16.0,31/12/2022 11:00:00,31/12/2022 09:00:00,Westerly Wind,Above freezing
214736,1672466040,1672458840,8.0,Mostly Cloudy,16.0,31/12/2022 12:00:00,31/12/2022 10:00:00,Westerly Wind,Above freezing


Execution Time: 1.17 seconds


<h3> 6.1.3 Comparing temperature differences with visibility ranges to classify weather severity</h3>

In [24]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
    {
        "$addFields": {
            "realfeel_difference": {"$subtract": ["$temp", "$wc"]},
            "visibility_status": {
                "$switch": {
                    "branches": [
                        {"case": {"$lt": ["$vis", 5]}, "then": "Low Visibility"},
                        {"case": {"$and": [{"$gte": ["$vis", 5]}, {"$lte": ["$vis", 15]}]}, "then": "Moderate Visibility"}
                    ],
                    "default": "High Visibility"
                }
            },
            "wind_chill_effect": {
                "$cond": {
                    "if": {"$gt": [{"$subtract": ["$temp", "$wc"]}, 20]},
                    "then": "Significant Wind Chill Effect",
                    "else": "Negligible Wind Chill Effect"
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "Unnamed: 0": 1,
            "valid_time_gmt": 1,
            "realfeel_difference": 1,
            "vis": 1,
            "visibility_status": 1,
            "wind_chill_effect": 1,
            "wx_phrase": 1,
            "valid_date_time": 1
        }
    }
]

# Run the aggregation query
data = db.weather.philadelphia.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")
#0.139 seconds
#.2645 seconds
"""{
    "_id": 0,
    "Unnamed: 0": 1,
    "valid_time_gmt": 1,
    "realfeel_difference": { "$subtract": ["$temp", "$wc"] },
    "vis": 1,
    "visibility_status": {
      "$switch": {
        "branches": [
          { "case": { "$lt": ["$vis", 5] }, "then": "Low Visibility" },
          { "case": { "$and": [{ "$gte": ["$vis", 5] }, { "$lte": ["$vis", 15] }] }, "then": "Moderate Visibility" }
        ],
        "default": "High Visibility"
      }
    },
    "wind_chill_effect": {
      "$cond": {
        "if": { "$gt": [{ "$subtract": ["$temp", "$wc"] }, 20] },
        "then": "Significant Wind Chill Effect",
        "else": "Negligible Wind Chill Effect"
      }
    },
    "wx_phrase": 1,
    "valid_date_time": 1
  }"""
#    "temp" - "wc" AS realfeel_difference,     "vis",     CASE         WHEN "vis" < 5 THEN 'Low Visibility'         WHEN "vis" BETWEEN 5 AND 15 THEN 'Moderate Visibility'         ELSE 'High Visibility'     END AS visibility_status,     CASE         WHEN "temp" - "wc" > 20 THEN 'Significant Wind Chill Effect'         ELSE 'Negligible Wind Chill Effect'     END AS wind_chill_effect,

,valid_time_gmt,wx_phrase,vis,valid_date_time,realfeel_difference,visibility_status,wind_chill_effect
0,1072932840,Fair,16.0,01/01/2004 11:00:00,5.0,High Visibility,Negligible Wind Chill Effect
1,1072936440,Fair,16.0,01/01/2004 12:00:00,4.0,High Visibility,Negligible Wind Chill Effect
2,1072940040,Fair,16.0,01/01/2004 13:00:00,4.0,High Visibility,Negligible Wind Chill Effect
3,1072943640,Fair,16.0,01/01/2004 14:00:00,4.0,High Visibility,Negligible Wind Chill Effect
4,1072947240,Fair,16.0,01/01/2004 15:00:00,4.0,High Visibility,Negligible Wind Chill Effect
...,...,...,...,...,...,...,...
214733,1672448040,Mostly Cloudy,16.0,31/12/2022 07:00:00,0.0,High Visibility,Negligible Wind Chill Effect
214734,1672451640,Mostly Cloudy,16.0,31/12/2022 08:00:00,0.0,High Visibility,Negligible Wind Chill Effect
214735,1672455240,Mostly Cloudy,16.0,31/12/2022 09:00:00,0.0,High Visibility,Negligible Wind Chill Effect
214736,1672458840,Mostly Cloudy,16.0,31/12/2022 10:00:00,0.0,High Visibility,Negligible Wind Chill Effect


Execution Time: 1.05 seconds


<h3> 6.1.4 Calculating wind direction variance and assigning weather status based on wind chill </h3>

In [7]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
    {
        "$addFields": {
            "opposite_wdir": {"$add": ["$wdir", 180]},
            "reduced_wind_chill": {"$subtract": ["$wc", 5]},
            "wind_temperature_factor": {"$multiply": ["$temp", "$wspd"]},
            "comfort_index": {"$divide": [{"$add": ["$temp", "$wspd"]}, 2]},
            "comfort_status": {
                "$switch": {
                    "branches": [
                        {"case": {"$gt": ["$temp", 85]}, "then": "Too Hot"},
                        {"case": {"$lt": ["$temp", 32]}, "then": "Too Cold"}
                    ],
                    "default": "Comfortable"
                }
            },
            "wind_chill_status": {
                "$switch": {
                    "branches": [
                        {"case": {"$lt": ["$wc", 0]}, "then": "Extreme Cold"},
                        {"case": {"$and": [{"$gte": ["$wc", 0]}, {"$lte": ["$wc", 32]}]}, "then": "Chilly"}
                    ],
                    "default": "Not Chilly"
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "Unnamed: 0": 1,
            "valid_time_gmt": 1,
            "opposite_wdir": 1,
            "reduced_wind_chill": 1,
            "wind_temperature_factor": 1,
            "comfort_index": 1,
            "comfort_status": 1,
            "wind_chill_status": 1,
            "wx_phrase": 1,
            "valid_date_time": 1
        }
    }
]
# Run the aggregation query
data = db.weather.philadelphia.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")
#0.142 seconds

,valid_time_gmt,wx_phrase,valid_date_time,opposite_wdir,reduced_wind_chill,wind_temperature_factor,comfort_index,comfort_status,wind_chill_status
0,1072932840,Fair,01/01/2004 11:00:00,410.0,-7.0,51.0,10.0,Too Cold,Extreme Cold
1,1072936440,Fair,01/01/2004 12:00:00,430.0,-8.0,11.0,6.0,Too Cold,Extreme Cold
2,1072940040,Fair,01/01/2004 13:00:00,420.0,-8.0,13.0,7.0,Too Cold,Extreme Cold
3,1072943640,Fair,01/01/2004 14:00:00,430.0,-9.0,0.0,6.5,Too Cold,Extreme Cold
4,1072947240,Fair,01/01/2004 15:00:00,420.0,-8.0,11.0,6.0,Too Cold,Extreme Cold
...,...,...,...,...,...,...,...,...,...
214733,1672448040,Mostly Cloudy,31/12/2022 07:00:00,300.0,1.0,36.0,6.0,Too Cold,Chilly
214734,1672451640,Mostly Cloudy,31/12/2022 08:00:00,NaN,1.0,0.0,3.0,Too Cold,Chilly
214735,1672455240,Mostly Cloudy,31/12/2022 09:00:00,NaN,2.0,0.0,3.5,Too Cold,Chilly
214736,1672458840,Mostly Cloudy,31/12/2022 10:00:00,NaN,3.0,0.0,4.0,Too Cold,Chilly


Execution Time: 1.23 seconds


<h2> 6.2.1 COUNT </h2>

In [15]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
  {
    "$group": {
      "_id": None,
      "Total_Observations": { "$sum": 1 },
      "High_Heat_Index_Count": {
        "$sum": {
          "$cond": [{ "$gt": ["$heat_index", 90] }, 1, 0]
        }
      },
      "High_Humidity_Count": {
        "$sum": {
          "$cond": [{ "$gte": ["$rh", 80] }, 1, 0]
        }
      },
      "Low_Visibility_Count": {
        "$sum": {
          "$cond": [{ "$lt": ["$vis", 2] }, 1, 0]
        }
      },
      "High_Pressure_Count": {
        "$sum": {
          "$cond": [{ "$gt": ["$pressure", 1020] }, 1, 0]
        }
      },
      "Extreme_Wind_Chill_Count": {
        "$sum": {
          "$cond": [{ "$lt": ["$wc", -20] }, 1, 0]
        }
      }
    }
  }
]

# Run the aggregation query
data = db.weather.philadelphia.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")

,_id,Total_Observations,High_Heat_Index_Count,High_Humidity_Count,Low_Visibility_Count,High_Pressure_Count,Extreme_Wind_Chill_Count
0,None,214738,0,80376,5176,62927,847


Execution Time: 0.19 seconds


<h2> 6.2.2 SUM</h2>

In [19]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
  {
    "$group": {
      "_id": None,
      "Total_Observations": { "$sum": 1 },
      "Total_Heat_Index_Over_90": {
        "$sum": {
          "$cond": [{ "$gt": ["$heat_index", 90] }, "$heat_index", 0]
        }
      },
      "Total_High_Humidity": {
        "$sum": {
          "$cond": [{ "$gte": ["$rh", 80] }, "$rh", 0]
        }
      },
      "Total_Low_Visibility_Events": {
        "$sum": {
          "$cond": [{ "$lt": ["$vis", 2] }, 1, 0]
        }
      },
      "Total_High_Pressure": {
        "$sum": {
          "$cond": [{ "$gt": ["$pressure", 1020] }, "$pressure", 0]
        }
      },
      "Total_Extreme_Wind_Chill_Events": {
        "$sum": {
          "$cond": [{ "$lt": ["$wc", -20] }, 1, 0]
        }
      }
    }
  }
]

# Run the aggregation query
data = db.weather.philadelphia.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.3f} seconds")

,_id,Total_Observations,Total_Heat_Index_Over_90,Total_High_Humidity,Total_Low_Visibility_Events,Total_High_Pressure,Total_Extreme_Wind_Chill_Events
0,None,214738,0,7140224.0,5176,64499437.52,847


Execution Time: 0.193 seconds


<h2> 6.2.4 MAX</h2>

In [20]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
  {
    "$group": {
      "_id": None,
      "Max_Temperature": { "$max": "$temp" },
      "Max_Wind_Speed_Over_10": {
        "$max": {
          "$cond": [{ "$gte": ["$wspd", 10] }, "$wspd", None]
        }
      },
      "Max_Heat_Index_Over_90": {
        "$max": {
          "$cond": [{ "$gt": ["$heat_index", 90] }, "$heat_index", None]
        }
      },
      "Max_Relative_Humidity_Over_80": {
        "$max": {
          "$cond": [{ "$gte": ["$rh", 80] }, "$rh", None]
        }
      },
      "Min_Visibility_Under_2": {
        "$max": {
          "$cond": [{ "$lt": ["$vis", 2] }, "$vis", None]
        }
      },
      "Max_Pressure_Over_1020": {
        "$max": {
          "$cond": [{ "$gt": ["$pressure", 1020] }, "$pressure", None]
        }
      },
      "Max_Wind_Chill": { "$max": "$wc" }
    }
  }
]

# Run the aggregation query
data = db.weather.philadelphia.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.3f} seconds")

,_id,Max_Temperature,Max_Wind_Speed_Over_10,Max_Heat_Index_Over_90,Max_Relative_Humidity_Over_80,Min_Visibility_Under_2,Max_Pressure_Over_1020,Max_Wind_Chill
0,None,40.0,185.0,None,100.0,1.0,1059.4,40.0


Execution Time: 0.236 seconds
